In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from __future__ import absolute_import
import os
import findspark
findspark.init(
    os.getenv("SPARK_HOME", "/Users/fitrakacamarga/.sdkman/candidates/spark/current")
)
import pyspark  # noqa
from pyspark.sql import SparkSession  # noqa

In [3]:
cur_dir = os.path.dirname(os.path.realpath("./"))
external_libs_dir = os.path.join(cur_dir, "signal", "lib", "external")
external_libs_jars = [
    os.path.join(external_libs_dir, f) for f in os.listdir(external_libs_dir)
]
spark_warehouse_dir, meta_dir = "/tmp/spark-warehouse", "/tmp/spark-meta"

In [4]:
spark = (
        SparkSession.builder.master("local")
        # workaround to avoid snappy library issue
        .config("spark.sql.parquet.compression.codec", "uncompressed")
        .config("spark.master", "local")
        .config("spark.driver.memory", "1G")
        # make spark-warehouse temporary
        .config("spark.sql.warehouse.dir", spark_warehouse_dir)
        # make metastore temporary
        .config(
            "spark.driver.extraJavaOptions", "-Dderby.system.home={}".format(meta_dir)
        )
        .config(
            "spark.jars.repositories",
            "http://packages.confluent.io/maven/,https://repo1.maven.org/maven2/",
        )
        .config("spark.jars", ",".join(external_libs_jars))
        .config(
            "spark.jars.packages",
            "org.apache.spark:spark-avro_2.12:3.5.3,za.co.absa:abris_2.12:6.4.0,com.lihaoyi:os-lib_2.12:0.8.1,org.apache.kafka:kafka-clients:3.8.0,io.delta:delta-spark_2.12:3.2.1,org.apache.iceberg:iceberg-aws-bundle:1.6.1,org.apache.hadoop:hadoop-aws:3.2.2,org.apache.hadoop:hadoop-client:3.2.2,org.apache.hadoop:hadoop-client-runtime:3.2.2",
        )
        .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
        .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9050")
        .config("spark.hadoop.fs.s3a.access.key", "admin")
        .config("spark.hadoop.fs.s3a.secret.key", "mta@fabric")
        .config("spark.hadoop.fs.s3a.path.style.access", "true") 
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
        .config("spark.ui.port","4050")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        .appName("seeknal-test")
        .enableHiveSupport()
        .getOrCreate()
    )

http://packages.confluent.io/maven/ added as a remote repository with the name: repo-1
https://repo1.maven.org/maven2/ added as a remote repository with the name: repo-2
Ivy Default Cache set to: /Users/fitrakacamarga/.ivy2/cache
The jars for the packages stored in: /Users/fitrakacamarga/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
za.co.absa#abris_2.12 added as a dependency
com.lihaoyi#os-lib_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
io.delta#delta-spark_2.12 added as a dependency
org.apache.iceberg#iceberg-aws-bundle added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.hadoop#hadoop-client added as a dependency
org.apache.hadoop#hadoop-client-runtime added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4584ca95-6d50-4e26-8bd8-6181bd36ca6d;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.5.3 in central
	found org.tukaani#xz;1.9 in local-m2-cache
	

:: loading settings :: url = jar:file:/Users/fitrakacamarga/.sdkman/candidates/spark/3.5.3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found com.fasterxml.jackson.core#jackson-core;2.12.3 in central
	found com.fasterxml.jackson.core#jackson-databind;2.10.5.1 in central
	found com.fasterxml.jackson.core#jackson-annotations;2.10.5 in central
	found org.apache.commons#commons-compress;1.21 in central
	found org.slf4j#slf4j-api;1.7.30 in spark-list
	found io.confluent#kafka-schema-serializer;6.2.1 in spark-list
	found io.confluent#kafka-schema-registry-client;6.2.1 in spark-list
	found jakarta.ws.rs#jakarta.ws.rs-api;2.1.6 in central
	found org.glassfish.jersey.core#jersey-common;2.34 in central
	found jakarta.annotation#jakarta.annotation-api;1.3.5 in central
	found org.glassfish.hk2.external#jakarta.inject;2.6.1 in central
	found org.glassfish.hk2#osgi-resource-locator;1.0.3 in central
	found io.swagger#swagger-annotations;1.6.2 in central
	found io.swagger#swagger-core;1.6.2 in central
	found org.apache.commons#commons-lang3;3.2.1 in central
	found com.fasterxml.jackson.dataformat#jackson-dataformat-yaml;2.10.5 in cen

In [4]:
from datetime import datetime, timedelta

from seeknal.entity import Entity
from seeknal.featurestore.feature_group import (
    FeatureGroup,
    Materialization,
    OfflineMaterialization,
    OfflineStore,
    OfflineStoreEnum,
    FeatureStoreFileOutput,
    OnlineStore,
    OnlineStoreEnum,
    HistoricalFeatures,
    FeatureLookup,
    FillNull,
    GetLatestTimeStrategy,
    OnlineFeatures,
)

from seeknal.flow import *
from seeknal.featurestore.featurestore import Feature
from seeknal.common_artifact import Source, Rule, Common, Dataset
from seeknal.project import Project
from seeknal.workspace import Workspace
from seeknal.tasks.sparkengine import aggregators as G
from seeknal.tasks.sparkengine import transformers as T
from seeknal.tasks.sparkengine.transformers.spark_engine_transformers import (
    JoinTablesByExpr,
    JoinType,
    TableJoinDef,
)
from pyspark import SparkContext
from pyspark.sql import DataFrame
import pandas as pd

In [5]:
from datetime import datetime, timedelta

from src.seeknal.entity import Entity
from src.seeknal.featurestore.feature_group import (
    FeatureGroup,
    Materialization,
    OfflineMaterialization,
    OfflineStore,
    OfflineStoreEnum,
    FeatureStoreFileOutput,
    OnlineStore,
    OnlineStoreEnum,
    HistoricalFeatures,
    FeatureLookup,
    FillNull,
    GetLatestTimeStrategy,
    OnlineFeatures,
)

from src.seeknal.flow import *
from src.seeknal.featurestore.featurestore import Feature
from src.seeknal.common_artifact import Source, Rule, Common, Dataset
from src.seeknal.project import Project
from src.seeknal.workspace import Workspace
from src.seeknal.tasks.sparkengine import aggregators as G
from src.seeknal.tasks.sparkengine import transformers as T
from src.seeknal.tasks.sparkengine.transformers.spark_engine_transformers import (
    JoinTablesByExpr,
    JoinType,
    TableJoinDef,
)
from pyspark import SparkContext
from pyspark.sql import DataFrame
import pandas as pd

# 1. Project and Workspace

To start working with seeknal, we need to set up a project and a workspace.

In [6]:
project = Project(name="test_project", description="test project")
# attach project
project.get_or_create()

Project(name='test_project', description='test project')

In [5]:
from seeknal.context import context

context.secrets 

Config({'SEEKNAL_USERNAME': 'fitra'})

In [8]:
# you can create different workspace under a project.
# in default, workspace is created under your account name.
Workspace()

Workspace(name='fitra', description='', private=False)

In [7]:
# create or use a workspace
Workspace(name="test_workspace", description="test workspace").get_or_create()

# check which workspace is active
Workspace.current()

WorkspaceTable(private=False, project_id=1, name='test_workspace', default_output_db=None, online_store_id=None, updated_at=datetime.datetime(2024, 12, 28, 21, 29, 54, 631670), description='test workspace', id=1, default_input_db=None, offline_store_id=None, created_at=datetime.datetime(2024, 12, 28, 21, 29, 54, 631423))

## 1.2 Add Common Articfacts 

In [10]:
# Add source
entity = Entity(name="subscriber", join_keys=["msisdn"])
entity.get_or_create()
source = Source(
    "my_user_stay",
    Dataset(table="user_stay", params={"dateCol": "date_id"}),
    description="user stay",
    entity=entity,
)
source.get_or_create()
Source.list()

[2024-12-30 16:53:42+0700] WARNING - seeknal | Using an existing Source.
[2024-12-30 16:53:42+0700] WARNING - seeknal | Using an existing Source.
| name         | description   | details                                                  | entity     |
|--------------|---------------|----------------------------------------------------------|------------|
| my_user_stay | user stay     | {'table': 'user_stay', 'params': {'dateCol': 'date_id'}} | subscriber |
| my_test_df   | just a dummy  | {'table': 'test_df', 'params': {'dateCol': 'date_id'}}   |            |


In [11]:
# Add rule
Rule(name="foo", value="bar", description="foo bar").get_or_create()
Rule.list()

[2024-12-30 16:53:44+0700] WARNING - seeknal | Using an existing Rule.
[2024-12-30 16:53:44+0700] WARNING - seeknal | Using an existing Rule.
| name   | value          | description   |
|--------|----------------|---------------|
| foo    | bar            | foo bar       |
| blah   | ['foo', 'bar'] |               |


In [12]:
# get common yaml

Rule(name="foo", value="bar", description="foo bar").get_or_create()
Rule(name="blah", value=["foo", "bar"]).get_or_create()
Source(
    "my_user_stay",
    Dataset(hive_table="user_stay", params={"dateCol": "date_id"}),
    description="user stay",
).get_or_create()
print(Common.as_yaml())

[2024-12-30 16:53:46+0700] WARNING - seeknal | Using an existing Rule.
[2024-12-30 16:53:46+0700] WARNING - seeknal | Using an existing Rule.
[2024-12-30 16:53:46+0700] WARNING - seeknal | Using an existing Rule.
[2024-12-30 16:53:46+0700] WARNING - seeknal | Using an existing Rule.
[2024-12-30 16:53:46+0700] WARNING - seeknal | Using an existing Source.
[2024-12-30 16:53:46+0700] WARNING - seeknal | Using an existing Source.
rules:
- id: foo
  rule:
    value: bar
- id: blah
  rule:
    value:
    - foo
    - bar
sources:
- id: my_user_stay
  params:
    dateCol: date_id
  table: user_stay
- id: my_test_df
  params:
    dateCol: date_id
  table: test_df



# 2. Tasks

DS2 build rich tasks that can be used to automate the process of data science.

## 2.1 Feature Engine Task

In [13]:
columns = "day:string, feature1:float, feature2:float, id:string"
vals = [
    ("20190620", 1.0, 1.0, "1"),
    ("20190610", -1.0, -1.0, "1"),
    ("20190602", 50.0, 50.0, "1"),
    ("20190601", 0.0, 0.0, "1"),
    ("20190520", 22.2, 22.2, "1"),
    ("20190510", 2.0, 2.0, "1"),
    ("20190501", 2.1, 2.1, "1"),
    ("20190620", 1.0, 1.0, "2"),
    ("20190710", None, None, "2"),
    ("20190602", 50.0, 50.0, "2"),
    ("20190601", 0.0, 0.0, "2"),
    ("20190520", 22.2, 22.2, "2"),
    ("20190510", 2.0, 2.0, "2"),
    ("20190501", 2.1, 2.1, "2"),
]

daily_features_1 = spark.createDataFrame(vals, columns)
daily_features_1.write.saveAsTable("test_df")

24/12/30 16:53:51 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/12/30 16:53:51 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/12/30 16:53:52 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
24/12/30 16:53:52 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore fitrakacamarga@192.168.50.152


AnalysisException: [TABLE_OR_VIEW_ALREADY_EXISTS] Cannot create table or view `spark_catalog`.`default`.`test_df` because it already exists.
Choose a different name, drop or replace the existing object, or add the IF NOT EXISTS clause to tolerate pre-existing objects.

In [14]:
yaml_str = """
pipeline:
  input:
    table: {}
  stages:
    - className: tech.mta.seeknal.transformers.SQL
      params:
        statement: >-
          SELECT day, CONCAT(id, "-append") as id, feature1, feature2 FROM __THIS__
""".format(
    "test_df"
)

res = SparkEngineTask().add_yaml(yaml_str).transform(spark)
res.show()

24/12/30 16:53:56 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+--------+--------+--------+--------+
|     day|      id|feature1|feature2|
+--------+--------+--------+--------+
|20190620|1-append|     1.0|     1.0|
|20190610|1-append|    -1.0|    -1.0|
|20190602|1-append|    50.0|    50.0|
|20190601|1-append|     0.0|     0.0|
|20190520|1-append|    22.2|    22.2|
|20190510|1-append|     2.0|     2.0|
|20190501|1-append|     2.1|     2.1|
|20190620|2-append|     1.0|     1.0|
|20190710|2-append|    NULL|    NULL|
|20190602|2-append|    50.0|    50.0|
|20190601|2-append|     0.0|     0.0|
|20190520|2-append|    22.2|    22.2|
|20190510|2-append|     2.0|     2.0|
|20190501|2-append|     2.1|     2.1|
+--------+--------+--------+--------+



In [8]:
Source(
    "my_test_df",
    Dataset(table="test_df", params={"dateCol": "date_id"}),
    description="just a dummy",
).get_or_create()

[2024-12-29 08:09:05+0700] WARNING - seeknal | Using an existing Source.


Source(name='my_test_df', details=Dataset(id=None, table='test_df', source=None, params={'dateCol': 'date_id'}, repartition=None, limit=None, partitions=None, path=None, connId=None), description='just a dummy', entity=None)

In [15]:
# transform data with common yaml
yaml_str = """
    pipeline:
      input:
        id: my_test_df
      stages:
        - className: tech.mta.seeknal.transformers.SQL
          params:
            statement: >-
              SELECT day, CONCAT(id, "-append") as id, feature1, feature2 FROM __THIS__
    """

res = (
        SparkEngineTask()
        .add_common_yaml(Common.as_yaml())
        .add_yaml(yaml_str)
        .transform(spark)
    )
res.show()


+--------+--------+--------+--------+
|     day|      id|feature1|feature2|
+--------+--------+--------+--------+
|20190620|1-append|     1.0|     1.0|
|20190610|1-append|    -1.0|    -1.0|
|20190602|1-append|    50.0|    50.0|
|20190601|1-append|     0.0|     0.0|
|20190520|1-append|    22.2|    22.2|
|20190510|1-append|     2.0|     2.0|
|20190501|1-append|     2.1|     2.1|
|20190620|2-append|     1.0|     1.0|
|20190710|2-append|    NULL|    NULL|
|20190602|2-append|    50.0|    50.0|
|20190601|2-append|     0.0|     0.0|
|20190520|2-append|    22.2|    22.2|
|20190510|2-append|     2.0|     2.0|
|20190501|2-append|     2.1|     2.1|
+--------+--------+--------+--------+



In [16]:
df = spark.table("test_df")
yaml_str = """
pipeline:
  stages:
    - className: tech.mta.seeknal.transformers.SQL
      params:
        statement: >-
          SELECT day, CONCAT(id, "-append") as id, feature1, feature2 FROM __THIS__
"""
res = (
    SparkEngineTask().add_yaml(yaml_str).add_input(dataframe=df).transform(spark)
)
res.show()

+--------+--------+--------+--------+
|     day|      id|feature1|feature2|
+--------+--------+--------+--------+
|20190620|1-append|     1.0|     1.0|
|20190610|1-append|    -1.0|    -1.0|
|20190602|1-append|    50.0|    50.0|
|20190601|1-append|     0.0|     0.0|
|20190520|1-append|    22.2|    22.2|
|20190510|1-append|     2.0|     2.0|
|20190501|1-append|     2.1|     2.1|
|20190620|2-append|     1.0|     1.0|
|20190710|2-append|    NULL|    NULL|
|20190602|2-append|    50.0|    50.0|
|20190601|2-append|     0.0|     0.0|
|20190520|2-append|    22.2|    22.2|
|20190510|2-append|     2.0|     2.0|
|20190501|2-append|     2.1|     2.1|
+--------+--------+--------+--------+



In [17]:
format_date = T.Transformer(
    T.ClassName.ADD_DATE,
    inputCol="day",
    outputCol="new_date",
    inputDateFormat="yyyyMMdd",
    outputDateFormat="yyyy-MM-dd",
)
res = (
    SparkEngineTask()
    .add_input(dataframe=df)
    .add_common_yaml(Common.as_yaml())
    .add_stage(
        transformer=T.SQL(
            statement="SELECT day, CONCAT(id, '-append') as id, feature1, feature2 FROM __THIS__"
        )
    )
    .add_stage(transformer=format_date)
    .transform(spark)
)
res.show()

+--------+--------+--------+--------+----------+
|     day|      id|feature1|feature2|  new_date|
+--------+--------+--------+--------+----------+
|20190620|1-append|     1.0|     1.0|2019-06-20|
|20190610|1-append|    -1.0|    -1.0|2019-06-10|
|20190602|1-append|    50.0|    50.0|2019-06-02|
|20190601|1-append|     0.0|     0.0|2019-06-01|
|20190520|1-append|    22.2|    22.2|2019-05-20|
|20190510|1-append|     2.0|     2.0|2019-05-10|
|20190501|1-append|     2.1|     2.1|2019-05-01|
|20190620|2-append|     1.0|     1.0|2019-06-20|
|20190710|2-append|    NULL|    NULL|2019-07-10|
|20190602|2-append|    50.0|    50.0|2019-06-02|
|20190601|2-append|     0.0|     0.0|2019-06-01|
|20190520|2-append|    22.2|    22.2|2019-05-20|
|20190510|2-append|     2.0|     2.0|2019-05-10|
|20190501|2-append|     2.1|     2.1|2019-05-01|
+--------+--------+--------+--------+----------+



In [18]:
df = spark.table("test_df")
res = (
    SparkEngineTask()
    .add_input(dataframe=df)
    .add_stage(
        transformer=T.SQL(
            statement="SELECT day, CONCAT(id, '-append') as id, feature1, feature2 FROM __THIS__"
        )
    )
    .transform(spark)
)

aggr = G.Aggregator(
    group_by_cols=["id"],
    aggregators=[
        G.FunctionAggregator(
            inputCol="feature1", outputCol="feature1_sum", accumulatorFunction="sum"
        ),
        G.ExpressionAggregator(
            outputCol="feature2_sum", expression="sum(feature2)"
        ),
    ],
)
res = (
    SparkEngineTask(name="create aggregates")
    .add_input(dataframe=df)
    .add_stage(aggregator=aggr)
    .add_output(
        source="file",
        params={"path": "{}/test_output".format("/tmp/seeknal_result"), "format": "parquet"},
    )
    .transform(spark, materialize=False)
)
res.show()

+---+-----------------+-----------------+
| id|     feature1_sum|     feature2_sum|
+---+-----------------+-----------------+
|  1|76.30000066757202|76.30000066757202|
|  2|77.30000066757202|77.30000066757202|
+---+-----------------+-----------------+



In [19]:
spine_columns = "msisdn:string, app_day:string, feature3:float"
spine_vals = [("1", "20190510", 1.0), ("2", "20190715", 2.0)]
spine_df = spark.createDataFrame(spine_vals, spine_columns)

tables = [
    TableJoinDef(
        table=spine_df,
        joinType=JoinType.LEFT,
        alias="b",
        joinExpression="a.id = b.msisdn",
    )
]
join = JoinTablesByExpr(tables=tables, select_stm="a.*, b.feature3, b.app_day")
res = (
    SparkEngineTask()
    .add_input(dataframe=spark.table("test_df"))
    .add_stage(transformer=join)
    .transform(spark)
)
res.show()

+--------+--------+--------+---+--------+--------+
|     day|feature1|feature2| id|feature3| app_day|
+--------+--------+--------+---+--------+--------+
|20190620|     1.0|     1.0|  1|     1.0|20190510|
|20190610|    -1.0|    -1.0|  1|     1.0|20190510|
|20190602|    50.0|    50.0|  1|     1.0|20190510|
|20190601|     0.0|     0.0|  1|     1.0|20190510|
|20190520|    22.2|    22.2|  1|     1.0|20190510|
|20190510|     2.0|     2.0|  1|     1.0|20190510|
|20190501|     2.1|     2.1|  1|     1.0|20190510|
|20190620|     1.0|     1.0|  2|     2.0|20190715|
|20190710|    NULL|    NULL|  2|     2.0|20190715|
|20190602|    50.0|    50.0|  2|     2.0|20190715|
|20190601|     0.0|     0.0|  2|     2.0|20190715|
|20190520|    22.2|    22.2|  2|     2.0|20190715|
|20190510|     2.0|     2.0|  2|     2.0|20190715|
|20190501|     2.1|     2.1|  2|     2.0|20190715|
+--------+--------+--------+---+--------+--------+



In [20]:
df = spark.table("test_df")
add_month = T.Transformer(
    T.ClassName.ADD_DATE,
    inputCol="day",
    outputCol="month",
    inputDateFormat="yyyyMMdd",
    outputDateFormat="yyyy-MM-01",
)
preprocess = (
    SparkEngineTask()
    .add_input(dataframe=df)
    .add_stage(
        transformer=T.SQL(
            statement="SELECT day, CONCAT(id, '-append') as id, feature1, feature2 FROM __THIS__"
        )
    )
)
res_one = preprocess.copy().add_stage(transformer=add_month).transform(spark)
res_two = (
    preprocess.copy()
    .add_sql("SELECT *, CONCAT(id, '-append-again') as id_b FROM __THIS__")
    .transform(spark)
)
res_one.show()
res_two.show()

+--------+--------+--------+--------+----------+
|     day|      id|feature1|feature2|     month|
+--------+--------+--------+--------+----------+
|20190620|1-append|     1.0|     1.0|2019-06-20|
|20190610|1-append|    -1.0|    -1.0|2019-06-10|
|20190602|1-append|    50.0|    50.0|2019-06-02|
|20190601|1-append|     0.0|     0.0|2019-06-01|
|20190520|1-append|    22.2|    22.2|2019-05-20|
|20190510|1-append|     2.0|     2.0|2019-05-10|
|20190501|1-append|     2.1|     2.1|2019-05-01|
|20190620|2-append|     1.0|     1.0|2019-06-20|
|20190710|2-append|    NULL|    NULL|2019-07-10|
|20190602|2-append|    50.0|    50.0|2019-06-02|
|20190601|2-append|     0.0|     0.0|2019-06-01|
|20190520|2-append|    22.2|    22.2|2019-05-20|
|20190510|2-append|     2.0|     2.0|2019-05-10|
|20190501|2-append|     2.1|     2.1|2019-05-01|
+--------+--------+--------+--------+----------+

+--------+--------+--------+--------+--------------------+
|     day|      id|feature1|feature2|                id_b|

In [21]:
df = spark.table("test_df")
preprocess = (
    SparkEngineTask()
    .add_input(dataframe=df)
    .add_stage(
        transformer=T.SQL(
            statement="SELECT day, CONCAT(id, '-append') as id, feature1, feature2 FROM __THIS__"
        )
    )
)

preprocess.update_stage(
    0,
    transformer=T.SQL(
        statement="SELECT *, CONCAT(id, '-append-again') as id_b FROM __THIS__"
    ),
)

res = preprocess.transform(spark)
res.show()

+--------+--------+--------+---+--------------+
|     day|feature1|feature2| id|          id_b|
+--------+--------+--------+---+--------------+
|20190620|     1.0|     1.0|  1|1-append-again|
|20190610|    -1.0|    -1.0|  1|1-append-again|
|20190602|    50.0|    50.0|  1|1-append-again|
|20190601|     0.0|     0.0|  1|1-append-again|
|20190520|    22.2|    22.2|  1|1-append-again|
|20190510|     2.0|     2.0|  1|1-append-again|
|20190501|     2.1|     2.1|  1|1-append-again|
|20190620|     1.0|     1.0|  2|2-append-again|
|20190710|    NULL|    NULL|  2|2-append-again|
|20190602|    50.0|    50.0|  2|2-append-again|
|20190601|     0.0|     0.0|  2|2-append-again|
|20190520|    22.2|    22.2|  2|2-append-again|
|20190510|     2.0|     2.0|  2|2-append-again|
|20190501|     2.1|     2.1|  2|2-append-again|
+--------+--------+--------+---+--------------+



In [22]:
df = spark.table("test_df")
preprocess = (
    SparkEngineTask()
    .add_input(dataframe=df)
    .add_stage(
        transformer=T.SQL(
            statement="SELECT day, CONCAT(id, '-append') as id, feature1, feature2 FROM __THIS__"
        )
    )
)
preprocess.insert_stage(
    0,
    transformer=T.SQL(
        statement="SELECT *, CONCAT(id, '-append-again') as id_b FROM __THIS__"
    ),
)
preprocess.print_yaml()

res = preprocess.transform(spark)
res.show()

pipeline:
  input: {}
  stages:
  - className: tech.mta.seeknal.transformers.SQL
    params:
      statement: SELECT *, CONCAT(id, '-append-again') as id_b FROM __THIS__
  - className: tech.mta.seeknal.transformers.SQL
    params:
      statement: SELECT day, CONCAT(id, '-append') as id, feature1, feature2 FROM
        __THIS__

+--------+--------+--------+--------+
|     day|      id|feature1|feature2|
+--------+--------+--------+--------+
|20190620|1-append|     1.0|     1.0|
|20190610|1-append|    -1.0|    -1.0|
|20190602|1-append|    50.0|    50.0|
|20190601|1-append|     0.0|     0.0|
|20190520|1-append|    22.2|    22.2|
|20190510|1-append|     2.0|     2.0|
|20190501|1-append|     2.1|     2.1|
|20190620|2-append|     1.0|     1.0|
|20190710|2-append|    NULL|    NULL|
|20190602|2-append|    50.0|    50.0|
|20190601|2-append|     0.0|     0.0|
|20190520|2-append|    22.2|    22.2|
|20190510|2-append|     2.0|     2.0|
|20190501|2-append|     2.1|     2.1|
+--------+--------+----

In [23]:
import datetime
df = spark.table("test_df")

res = (
    SparkEngineTask()
    .add_input(dataframe=df)
    .set_date_col("day")
    .add_stage(
        transformer=T.SQL(
            statement="SELECT day, CONCAT(id, '-append') as id, feature1, feature2 FROM __THIS__"
        )
    )
    .transform(
        start_date=datetime.datetime(2019, 5, 1),
        end_date=datetime.datetime(2019, 5, 10),
    )
)
res.show()

+--------+--------+--------+--------+
|     day|      id|feature1|feature2|
+--------+--------+--------+--------+
|20190510|1-append|     2.0|     2.0|
|20190501|1-append|     2.1|     2.1|
|20190510|2-append|     2.0|     2.0|
|20190501|2-append|     2.1|     2.1|
+--------+--------+--------+--------+



In [24]:
from datetime import datetime
res = (
        SparkEngineTask()
        .add_input(dataframe=df)
        .set_date_col("day")
        .get_date_available(after_date=datetime(2019, 5, 10))
)
print(res)

['20190510', '20190520', '20190601', '20190602', '20190610', '20190620', '20190710']


## 2.3 DuckDB Task

In [26]:
arrow_df = pq.read_table("src/tests/data/poi_sample.parquet")
my_duckdb = (
    DuckDBTask()
    .add_input(dataframe=arrow_df)
    .add_sql("SELECT poi_name, lat, long FROM __THIS__")
    .add_sql("SELECT poi_name, lat FROM __THIS__")
)
print(my_duckdb.__dict__)
res = my_duckdb.transform()

print(res)

{'is_spark_job': False, 'kind': 'DuckDBTask', 'name': None, 'description': None, 'common': None, 'input': {'dataframe': pyarrow.Table
poi_id: string
poi_name: string
category_level1_name: string
category_level2_name: string
category_level3_name: string
territory_level2: string
territory_level3: string
territory_level4: string
territory_level5: string
lat: double
long: double
----
poi_id: [["458w281x-608b10bc6f404a449ebf231761be1778","458w281x-9ee2bb0a12ca4cd580ad92d2d9262399","458w281x-1fcc6faea91d4b9696af2907156a324f","458w281x-d557af639fad4cb091de177eefad4e13","458w281x-e69ec25912bd49d78700adfb0de423e6",...,"458w281m-11ced1f2d96f4c1ea6128dbe2ba1bb33","458w281m-71a65a050f0d42fa8cff96faa6912a35","458w281k-284dbd33875b40bdb395bf9d6ea80aa8","458w281m-2aeab547e7d942c992836a0341366eb1","458w281m-e84be7db378c443ea485578cac98e724"]]
poi_name: [["Car Wash PETRONAS","Maplewood Saujana","Dietetics & Food Services","Facility Maintenance & Biomedical","Bhpetrol",...,"RCA Consortium Sdn. Bhd.","Gl

In [9]:
my_duckdb = (
    DuckDBTask()
    .add_input(path="seeknal/tests/data/poi_sample.parquet/*.parquet")
    .add_sql("SELECT poi_name, lat, long FROM __THIS__")
    .add_sql("SELECT poi_name, lat FROM __THIS__")
)
print(my_duckdb.__dict__)
res = my_duckdb.transform()

print(res)

{'is_spark_job': False, 'kind': 'DuckDBTask', 'name': None, 'description': None, 'common': None, 'input': {'path': 'seeknal/tests/data/poi_sample.parquet/*.parquet'}, 'stages': [{'sql': 'SELECT poi_name, lat, long FROM __THIS__'}, {'sql': 'SELECT poi_name, lat FROM __THIS__'}], 'output': None, 'date': None}
pyarrow.Table
poi_name: string
lat: double
----
poi_name: [["Car Wash PETRONAS","Maplewood Saujana","Dietetics & Food Services","Facility Maintenance & Biomedical","Bhpetrol",...,"RCA Consortium Sdn. Bhd.","Glow&Glamour Slimming","Bank Kerjasama Rakyat Malaysia Berhad","Pizza Hut","DMC Sticker"]]
lat: [[3.11663,3.11375,3.11443,3.11443,3.1141,...,3.0609,3.06254,3.06266,3.06255,3.06644]]


# 3. Flow

In [8]:
from seeknal.flow import (
    Flow,
    FlowInput,
    FlowOutput,
    FlowInputEnum,
    FlowOutputEnum,
    run_flow,
)

In [9]:

columns = "msisdn:string, lat:double, lon:double, start_time:string, end_time:string, count_hours:double, radius:double, movement_type:string, day:string"
vals = [
        (
            "id1",
            3.1165,
            101.5663,
            "2019-01-01 06:00:00",
            "2019-01-01 07:00:00",
            1.0,
            1395.04,
            "stay",
            "20190101",
        ),
        (
            "id2",
            3.812033,
            103.324633,
            "2019-01-01 06:00:00",
            "2019-01-01 07:00:00",
            1.0,
            841.36,
            "stay",
            "20190101",
        ),
        (
            "id3",
            3.0637,
            101.47016,
            "2019-01-01 06:00:00",
            "2019-01-01 07:00:00",
            1.0,
            1387.35,
            "stay",
            "20190101",
        ),
        (
            "id1",
            3.1186,
            101.6639,
            "2019-01-01 07:00:00",
            "2019-01-01 08:00:00",
            1.0,
            1234.22,
            "stay",
            "20190101",
        ),
        (
            "id1",
            3.1165,
            101.5663,
            "2019-01-01 06:00:00",
            "2019-01-01 07:00:00",
            1.0,
            1395.04,
            "stay",
            "20190102",
        ),
        (
            "id2",
            3.812033,
            103.324633,
            "2019-01-01 06:00:00",
            "2019-01-01 07:00:00",
            1.0,
            841.36,
            "stay",
            "20190102",
        ),
        (
            "id3",
            3.0637,
            101.47016,
            "2019-01-01 06:00:00",
            "2019-01-01 07:00:00",
            1.0,
            1387.35,
            "stay",
            "20190102",
        ),
        (
            "id1",
            3.1186,
            101.6639,
            "2019-01-01 07:00:00",
            "2019-01-01 08:00:00",
            1.0,
            1234.22,
            "stay",
            "20190102",
        ),
        (
            "id1",
            3.1165,
            101.5663,
            "2019-01-01 06:00:00",
            "2019-01-01 07:00:00",
            1.0,
            1395.04,
            "stay",
            "20190105",
        ),
        (
            "id2",
            3.812033,
            103.324633,
            "2019-01-01 06:00:00",
            "2019-01-01 07:00:00",
            1.0,
            841.36,
            "stay",
            "20190105",
        ),
        (
            "id3",
            3.0637,
            101.47016,
            "2019-01-01 06:00:00",
            "2019-01-01 07:00:00",
            1.0,
            1387.35,
            "stay",
            "20190105",
        ),
        (
            "id1",
            3.1186,
            101.6639,
            "2019-01-01 07:00:00",
            "2019-01-01 08:00:00",
            1.0,
            1234.22,
            "stay",
            "20190105",
        ),
    ]
user_stay = spark.createDataFrame(vals, columns)
(
    SparkEngineTask()
    .add_input(dataframe=user_stay)
    .transform()
    .write.mode("overwrite")
    .saveAsTable("user_stay")
)


24/12/30 16:57:36 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [29]:
user_stay.show()

+------+--------+----------+-------------------+-------------------+-----------+-------+-------------+--------+
|msisdn|     lat|       lon|         start_time|           end_time|count_hours| radius|movement_type|     day|
+------+--------+----------+-------------------+-------------------+-----------+-------+-------------+--------+
|   id1|  3.1165|  101.5663|2019-01-01 06:00:00|2019-01-01 07:00:00|        1.0|1395.04|         stay|20190101|
|   id2|3.812033|103.324633|2019-01-01 06:00:00|2019-01-01 07:00:00|        1.0| 841.36|         stay|20190101|
|   id3|  3.0637| 101.47016|2019-01-01 06:00:00|2019-01-01 07:00:00|        1.0|1387.35|         stay|20190101|
|   id1|  3.1186|  101.6639|2019-01-01 07:00:00|2019-01-01 08:00:00|        1.0|1234.22|         stay|20190101|
|   id1|  3.1165|  101.5663|2019-01-01 06:00:00|2019-01-01 07:00:00|        1.0|1395.04|         stay|20190102|
|   id2|3.812033|103.324633|2019-01-01 06:00:00|2019-01-01 07:00:00|        1.0| 841.36|         stay|20

In [36]:
context.project_id

1

In [23]:
from src.seeknal.context import context, require_project, get_workspace_id, check_project_id, get_project_id, check_workspace_id

In [31]:
project = Project(name="test_project", description="test project")
# attach project
project.get_or_create()

Project(name='test_project', description='test project')

In [33]:
Flow.list()

ProjectNotSetError: Project hasn't been set in your environment. Please set it with Project(name='...').

In [32]:
flow_input = FlowInput(kind=FlowInputEnum.HIVE_TABLE, value="user_stay")
flow_output = FlowOutput(kind=FlowOutputEnum.SPARK_DATAFRAME)

task_two = SparkEngineTask().add_sql("SELECT * FROM __THIS__")
task_three = DuckDBTask().add_sql("SELECT msisdn, lat, lon, movement_type, day FROM __THIS__")
flow = Flow(
    name="my_flow",
    input=flow_input,
    tasks=[task_two, task_three],
    output=FlowOutput(),
)

flow.get_or_create()


ProjectNotSetError: Project hasn't been set in your environment. Please set it with Project(name='...').

In [12]:
# save from the object
print(flow.run())

pyarrow.Table
msisdn: string
lat: double
lon: double
movement_type: string
day: string
----
msisdn: [["id1","id2","id3","id1","id1",...,"id1","id1","id2","id3","id1"]]
lat: [[3.1165,3.812033,3.0637,3.1186,3.1165,...,3.1186,3.1165,3.812033,3.0637,3.1186]]
lon: [[101.5663,103.324633,101.47016,101.6639,101.5663,...,101.6639,101.5663,103.324633,101.47016,101.6639]]
movement_type: [["stay","stay","stay","stay","stay",...,"stay","stay","stay","stay","stay"]]
day: [["20190101","20190101","20190101","20190101","20190102",...,"20190102","20190105","20190105","20190105","20190105"]]


In [13]:
# load from seeknal
flowtwo = Flow(name="my_flow").get_or_create()
print(flowtwo.run())

[2024-12-29 12:07:11+0700] WARNING - seeknal | Using an existing Flow.
pyarrow.Table
msisdn: string
lat: double
lon: double
movement_type: string
day: string
----
msisdn: [["id1","id2","id3","id1","id1",...,"id1","id1","id2","id3","id1"]]
lat: [[3.1165,3.812033,3.0637,3.1186,3.1165,...,3.1186,3.1165,3.812033,3.0637,3.1186]]
lon: [[101.5663,103.324633,101.47016,101.6639,101.5663,...,101.6639,101.5663,103.324633,101.47016,101.6639]]
movement_type: [["stay","stay","stay","stay","stay",...,"stay","stay","stay","stay","stay"]]
day: [["20190101","20190101","20190101","20190101","20190102",...,"20190102","20190105","20190105","20190105","20190105"]]


In [16]:
flow.as_dict()

{'name': 'my_flow',
 'input': {'value': 'user_stay', 'kind': 'hive_table'},
 'input_date_col': None,
 'tasks': [{'is_spark_job': True,
   'kind': 'SparkEngineTask',
   'name': None,
   'description': None,
   'common': 'rules:\n- id: foo\n  rule:\n    value: bar\n- id: blah\n  rule:\n    value:\n    - foo\n    - bar\nsources:\n- id: my_user_stay\n  params:\n    dateCol: date_id\n  table: user_stay\n- id: my_test_df\n  params:\n    dateCol: date_id\n  table: test_df\n',
   'input': None,
   'stages': [{'className': 'tech.mta.signal.transformers.SQL',
     'params': {'statement': 'SELECT * FROM __THIS__'}}],
   'output': None,
   'date': None,
   'feature': None,
   'default_input_db': 'base_input',
   'default_output_db': 'base_output',
   '_materialize': False,
   'class_name': 'seeknal.tasks.sparkengine.sparkengine.SparkEngineTask'},
  {'is_spark_job': False,
   'kind': 'DuckDBTask',
   'name': None,
   'description': None,
   'common': None,
   'input': None,
   'stages': [{'sql': 'S

In [17]:
my_flow = Flow.from_dict(flow.as_dict())
my_flow.run()

pyarrow.Table
msisdn: string
lat: double
lon: double
movement_type: string
day: string
----
msisdn: [["id1","id2","id3","id1","id1",...,"id1","id1","id2","id3","id1"]]
lat: [[3.1165,3.812033,3.0637,3.1186,3.1165,...,3.1186,3.1165,3.812033,3.0637,3.1186]]
lon: [[101.5663,103.324633,101.47016,101.6639,101.5663,...,101.6639,101.5663,103.324633,101.47016,101.6639]]
movement_type: [["stay","stay","stay","stay","stay",...,"stay","stay","stay","stay","stay"]]
day: [["20190101","20190101","20190101","20190101","20190102",...,"20190102","20190105","20190105","20190105","20190105"]]

# 4. Feature Group 

In [7]:
comm_day = spark.read.format("parquet").load("seeknal/tests/data/feateng_comm_day")
comm_day.write.saveAsTable("comm_day")

24/12/29 12:16:55 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/12/29 12:16:55 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/12/29 12:16:56 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
24/12/29 12:16:56 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore fitrakacamarga@192.168.50.152


AnalysisException: [TABLE_OR_VIEW_ALREADY_EXISTS] Cannot create table or view `spark_catalog`.`default`.`comm_day` because it already exists.
Choose a different name, drop or replace the existing object, or add the IF NOT EXISTS clause to tolerate pre-existing objects.

In [8]:
spark.table("comm_day").show()

24/12/30 15:37:04 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/12/30 15:37:04 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/12/30 15:37:06 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
24/12/30 15:37:06 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore fitrakacamarga@192.168.50.152
24/12/30 15:37:08 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------------+-------------------+---------------------+-----------------+------------------+--------------------+---------------------+----------------------+------------------------+-------------------------+--------------------------+----------------------------+------------------------+-------------------------+---------------------------+----------------------------+-----------------------------+-------------------------------+---------------------+----------------------+------------------------+-------------------------+------------------------+--------------------------+-------------------------+----------------------------+---------------------------+-----------------------+------------------------+----------------------+-----------------------+--------------------------+---------------------------+----------+----------+
|comm_count_call_in|comm_count_call_out|comm_count_call_inout|comm_count_sms_in|comm_count_sms_out|comm_count_sms_inout|comm_count_callsms_in|comm_count_

In [11]:
spark.sql("DROP TABLE seeknal.fg_1__1")

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `spark_catalog`.`seeknal`.`fg_1__1` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.

In [9]:
FeatureStoreFileOutput

seeknal.featurestore.featurestore.FeatureStoreFileOutput

In [10]:
materialization = Materialization(event_time_col="day", 
offline_materialization=OfflineMaterialization(
    store=OfflineStore(kind=OfflineStoreEnum.FILE, 
                       name="object_storage",
                       value=FeatureStoreFileOutput(path="s3a://warehouse/feature_store")), 
                       mode="overwrite", ttl=None),
    offline=True)

In [11]:
flow_input = FlowInput(kind=FlowInputEnum.HIVE_TABLE, value="comm_day")
my_flow = Flow(
    input=flow_input, tasks=None, output=FlowOutput(), name="my_flow_for_fg"
)

my_fg_one = FeatureGroup(
    name="comm_day_three",
    entity=Entity(name="msisdn", join_keys=["msisdn"]).get_or_create(),
    materialization=materialization,
).set_flow(my_flow)
my_fg_one.set_features()
# print(my_fg)
my_fg_one.get_or_create()

[2024-12-30 15:42:12+0700] WARNING - seeknal | Using an existing Flow.
[2024-12-30 15:42:12+0700] WARNING - seeknal | Using an existing Flow.


24/12/30 15:42:12 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


[2024-12-30 15:42:13+0700] WARNING - seeknal | Feature group already exists. Loading the feature group.
[2024-12-30 15:42:13+0700] WARNING - seeknal | Feature group already exists. Loading the feature group.


/Users/fitrakacamarga/project/mta/signal/.venv/lib/python3.11/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue: Expected `str` but got `dict` with value `{'path': 's3a://warehouse...store', 'kind': 'delta'}` - serialized value may not be as expected
  PydanticSerializationUnexpectedValue: Expected `FeatureStoreFileOutput` but got `dict` with value `{'path': 's3a://warehouse...store', 'kind': 'delta'}` - serialized value may not be as expected
  PydanticSerializationUnexpectedValue: Expected `FeatureStoreHiveTableOutput` but got `dict` with value `{'path': 's3a://warehouse...store', 'kind': 'delta'}` - serialized value may not be as expected
  Expected `enum` but got `str` with value `'file'` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


[2024-12-30 15:42:13+0700] WARNING - seeknal | Using an existing Flow.
[2024-12-30 15:42:13+0700] WARNING - seeknal | Using an existing Flow.


FeatureGroup(name='comm_day_three', entity=Entity(name='msisdn', join_keys=['msisdn'], pii_keys=None, description=None), id=3, materialization=Materialization(event_time_col='day', date_pattern=None, offline=True, online=False, offline_materialization=OfflineMaterialization(store=OfflineStore(value=FeatureStoreFileOutput(path='s3a://warehouse/feature_store', kind=<FileKindEnum.DELTA: 'delta'>), kind='file', name='object_storage'), mode='overwrite', ttl=None), online_materialization=OnlineMaterialization(store=OnlineStore(value=None, kind='file', name=None), ttl=1440)), source=Flow(name='my_flow_for_fg', input=FlowInput(value='comm_day', kind=<FlowInputEnum.HIVE_TABLE: 'hive_table'>), input_date_col=None, tasks=None, output=FlowOutput(value=None, kind=<FlowOutputEnum.SPARK_DATAFRAME: 'spark_dataframe'>), description=''), description=None, features=[Feature(name='comm_count_call_in', feature_id='67', description='', data_type='long', online_data_type='long', created_at='2024-12-30 02:51:

In [12]:
from datetime import datetime
my_fg_one.write(
        feature_start_time=datetime(2019, 3, 5)
    )

[2024-12-30 15:43:48+0700] INFO - seeknal | writing to offline-store
[2024-12-30 15:43:48+0700] INFO - seeknal | writing to offline-store


24/12/30 15:43:49 INFO AvroSerDe: Avro schema inferred from dataframe: {"type":"record","name":"topLevelRecord","fields":[{"name":"comm_count_call_in","type":["long","null"]},{"name":"comm_count_call_out","type":["long","null"]},{"name":"comm_count_call_inout","type":["long","null"]},{"name":"comm_count_sms_in","type":["long","null"]},{"name":"comm_count_sms_out","type":["long","null"]},{"name":"comm_count_sms_inout","type":["long","null"]},{"name":"comm_count_callsms_in","type":["long","null"]},{"name":"comm_count_callsms_out","type":["long","null"]},{"name":"comm_count_callsms_inout","type":["long","null"]},{"name":"comm_roamingcount_call_in","type":["long","null"]},{"name":"comm_roamingcount_call_out","type":["long","null"]},{"name":"comm_roamingcount_call_inout","type":["long","null"]},{"name":"comm_roamingcount_sms_in","type":["long","null"]},{"name":"comm_roamingcount_sms_out","type":["long","null"]},{"name":"comm_roamingcount_sms_inout","type":["long","null"]},{"name":"comm_roam

In [7]:
flow_input = FlowInput(kind=FlowInputEnum.HIVE_TABLE, value="comm_day")
my_flow = Flow(
    input=flow_input, tasks=None, output=FlowOutput(), name="my_flow_for_fg"
)

my_fg_one = FeatureGroup(
    name="comm_day",
    entity=Entity(name="msisdn", join_keys=["msisdn"]).get_or_create(),
    materialization=Materialization(event_time_col="day"),
).set_flow(my_flow)
my_fg_one.set_features()
# print(my_fg)
my_fg_one.get_or_create()

[2024-12-30 02:02:20+0700] WARNING - seeknal | Using an existing Flow.


24/12/30 02:02:21 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/12/30 02:02:21 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/12/30 02:02:22 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
24/12/30 02:02:22 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore fitrakacamarga@192.168.50.119
24/12/30 02:02:23 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


[2024-12-30 02:02:24+0700] WARNING - seeknal | Feature group already exists. Loading the feature group.
[2024-12-30 02:02:24+0700] WARNING - seeknal | Using an existing Flow.


FeatureGroup(name='comm_day', entity=Entity(name='msisdn', join_keys=['msisdn'], pii_keys=None, description=None), id=1, materialization=Materialization(event_time_col='day', date_pattern=None, offline=True, online=False, offline_materialization=OfflineMaterialization(store=OfflineStore(value=None, kind='hive_table', name='default'), mode='overwrite', ttl=None), online_materialization=OnlineMaterialization(store=OnlineStore(value=None, kind='file', name=None), ttl=1440)), source=Flow(name='my_flow_for_fg', input=FlowInput(value='comm_day', kind=<FlowInputEnum.HIVE_TABLE: 'hive_table'>), input_date_col=None, tasks=None, output=FlowOutput(value=None, kind=<FlowOutputEnum.SPARK_DATAFRAME: 'spark_dataframe'>), description=''), description=None, features=[Feature(name='comm_count_call_in', feature_id='1', description='', data_type='long', online_data_type='long', created_at='2024-12-29 12:09:58', updated_at='2024-12-29 12:09:58'), Feature(name='comm_count_call_out', feature_id='2', descript

In [9]:
my_fg_one

FeatureGroup(name='comm_day', entity=Entity(name='msisdn', join_keys=['msisdn'], pii_keys=None, description=None), id=1, materialization=Materialization(event_time_col='day', date_pattern=None, offline=True, online=False, offline_materialization=OfflineMaterialization(store=OfflineStore(value=None, kind='hive_table', name='default'), mode='overwrite', ttl=None), online_materialization=OnlineMaterialization(store=OnlineStore(value=None, kind='file', name=None), ttl=1440)), source=Flow(name='my_flow_for_fg', input=FlowInput(value='comm_day', kind=<FlowInputEnum.HIVE_TABLE: 'hive_table'>), input_date_col=None, tasks=None, output=FlowOutput(value=None, kind=<FlowOutputEnum.SPARK_DATAFRAME: 'spark_dataframe'>), description=''), description=None, features=[Feature(name='comm_count_call_in', feature_id='1', description='', data_type='long', online_data_type='long', created_at='2024-12-29 12:09:58', updated_at='2024-12-29 12:09:58'), Feature(name='comm_count_call_out', feature_id='2', descript

In [13]:
input_df = spark.read.table("comm_day")
my_fg_two = FeatureGroup(
    name="comm_day_two",
    entity=Entity(name="msisdn", join_keys=["msisdn"]).get_or_create(),
    materialization=Materialization(event_time_col="day"),
).set_dataframe(dataframe=input_df)

my_fg_two.set_features()
my_fg_two.get_or_create()

[2024-12-30 15:45:01+0700] WARNING - seeknal | Feature group already exists. Loading the feature group.
[2024-12-30 15:45:01+0700] WARNING - seeknal | Feature group already exists. Loading the feature group.


FeatureGroup(name='comm_day_two', entity=Entity(name='msisdn', join_keys=['msisdn'], pii_keys=None, description=None), id=2, materialization=Materialization(event_time_col='day', date_pattern=None, offline=True, online=False, offline_materialization=OfflineMaterialization(store=OfflineStore(value=None, kind='hive_table', name='default'), mode='overwrite', ttl=None), online_materialization=OnlineMaterialization(store=OnlineStore(value=None, kind='file', name=None), ttl=1440)), source=None, description=None, features=[Feature(name='comm_count_call_in', feature_id='34', description='', data_type='long', online_data_type='long', created_at='2024-12-29 12:10:27', updated_at='2024-12-29 12:10:27'), Feature(name='comm_count_call_out', feature_id='35', description='', data_type='long', online_data_type='long', created_at='2024-12-29 12:10:28', updated_at='2024-12-29 12:10:28'), Feature(name='comm_count_call_inout', feature_id='36', description='', data_type='long', online_data_type='long', cre

In [8]:
from datetime import datetime
my_fg_one.write(
        feature_start_time=datetime(2019, 3, 5)
    )

[2024-12-30 02:02:33+0700] INFO - seeknal | writing to offline-store


24/12/30 02:02:34 INFO AvroSerDe: Avro schema inferred from dataframe: {"type":"record","name":"topLevelRecord","fields":[{"name":"comm_count_call_in","type":["long","null"]},{"name":"comm_count_call_out","type":["long","null"]},{"name":"comm_count_call_inout","type":["long","null"]},{"name":"comm_count_sms_in","type":["long","null"]},{"name":"comm_count_sms_out","type":["long","null"]},{"name":"comm_count_sms_inout","type":["long","null"]},{"name":"comm_count_callsms_in","type":["long","null"]},{"name":"comm_count_callsms_out","type":["long","null"]},{"name":"comm_count_callsms_inout","type":["long","null"]},{"name":"comm_roamingcount_call_in","type":["long","null"]},{"name":"comm_roamingcount_call_out","type":["long","null"]},{"name":"comm_roamingcount_call_inout","type":["long","null"]},{"name":"comm_roamingcount_sms_in","type":["long","null"]},{"name":"comm_roamingcount_sms_out","type":["long","null"]},{"name":"comm_roamingcount_sms_inout","type":["long","null"]},{"name":"comm_roam

In [14]:
my_fg = FeatureGroup(name="comm_day_two").get_or_create()
my_fg.set_dataframe(dataframe=spark.read.table("comm_day")).write(
    feature_start_time=datetime(2019, 3, 5)
)

[2024-12-30 15:45:19+0700] WARNING - seeknal | Feature group already exists. Loading the feature group.
[2024-12-30 15:45:19+0700] WARNING - seeknal | Feature group already exists. Loading the feature group.
[2024-12-30 15:45:20+0700] INFO - seeknal | writing to offline-store
[2024-12-30 15:45:20+0700] INFO - seeknal | writing to offline-store


24/12/30 15:45:21 INFO AvroSerDe: Avro schema inferred from dataframe: {"type":"record","name":"topLevelRecord","fields":[{"name":"comm_count_call_in","type":["long","null"]},{"name":"comm_count_call_out","type":["long","null"]},{"name":"comm_count_call_inout","type":["long","null"]},{"name":"comm_count_sms_in","type":["long","null"]},{"name":"comm_count_sms_out","type":["long","null"]},{"name":"comm_count_sms_inout","type":["long","null"]},{"name":"comm_count_callsms_in","type":["long","null"]},{"name":"comm_count_callsms_out","type":["long","null"]},{"name":"comm_count_callsms_inout","type":["long","null"]},{"name":"comm_roamingcount_call_in","type":["long","null"]},{"name":"comm_roamingcount_call_out","type":["long","null"]},{"name":"comm_roamingcount_call_inout","type":["long","null"]},{"name":"comm_roamingcount_sms_in","type":["long","null"]},{"name":"comm_roamingcount_sms_out","type":["long","null"]},{"name":"comm_roamingcount_sms_inout","type":["long","null"]},{"name":"comm_roam

## 4.1 Historical Features

In [15]:
my_fg = FeatureGroup(name="comm_day_two")
fs = FeatureLookup(source=my_fg)
fillnull = FillNull(value="0.0", dataType="double")
hist = HistoricalFeatures(lookups=[fs], fill_nulls=[fillnull])
df = hist.to_dataframe(feature_start_time=datetime(2019, 3, 5))
df.show()

[2024-12-30 15:47:57+0700] WARNING - seeknal | Feature group already exists. Loading the feature group.
[2024-12-30 15:47:57+0700] WARNING - seeknal | Feature group already exists. Loading the feature group.


+----------+--------------------+-------------------+------------------+-------------------+---------------------+-----------------+------------------+--------------------+---------------------+----------------------+------------------------+-------------------------+--------------------------+----------------------------+------------------------+-------------------------+---------------------------+----------------------------+-----------------------------+-------------------------------+---------------------+----------------------+------------------------+-------------------------+------------------------+--------------------------+-------------------------+----------------------------+---------------------------+-----------------------+------------------------+----------------------+-----------------------+--------------------------+---------------------------+
|    msisdn|              __pk__|         event_time|comm_count_call_in|comm_count_call_out|comm_count_call_inout|comm_coun

In [18]:
my_fg = FeatureGroup(name="comm_day_three")
my_fg_two = FeatureGroup(name="comm_day_three")
fs = FeatureLookup(source=my_fg)
fs_two = FeatureLookup(source=my_fg_two, features=["comm_count_call_in"])
fillnull = FillNull(value="0.0", dataType="double")
hist = HistoricalFeatures(lookups=[fs], fill_nulls=[fillnull])
df = hist.to_dataframe(feature_start_time=datetime(2019, 3, 5))
df.show()

[2024-12-30 15:52:17+0700] WARNING - seeknal | Feature group already exists. Loading the feature group.
[2024-12-30 15:52:17+0700] WARNING - seeknal | Feature group already exists. Loading the feature group.
[2024-12-30 15:52:17+0700] WARNING - seeknal | Using an existing Flow.
[2024-12-30 15:52:17+0700] WARNING - seeknal | Using an existing Flow.


+----------+--------------------+-------------------+------------------+-------------------+---------------------+-----------------+------------------+--------------------+---------------------+----------------------+------------------------+-------------------------+--------------------------+----------------------------+------------------------+-------------------------+---------------------------+----------------------------+-----------------------------+-------------------------------+---------------------+----------------------+------------------------+-------------------------+------------------------+--------------------------+-------------------------+----------------------------+---------------------------+-----------------------+------------------------+----------------------+-----------------------+--------------------------+---------------------------+
|    msisdn|              __pk__|         event_time|comm_count_call_in|comm_count_call_out|comm_count_call_inout|comm_coun

In [10]:
my_fg

FeatureGroup(name='comm_day_three', entity=Entity(name='msisdn', join_keys=['msisdn'], pii_keys=None, description=None), id=3, materialization=Materialization(event_time_col='day', date_pattern=None, offline=True, online=False, offline_materialization=OfflineMaterialization(store=OfflineStore(value=FeatureStoreFileOutput(path='s3a://warehouse/feature_store', kind=<FileKindEnum.DELTA: 'delta'>), kind='file', name='object_storage'), mode='overwrite', ttl=None), online_materialization=OnlineMaterialization(store=OnlineStore(value=None, kind='file', name=None), ttl=1440)), source=Flow(name='my_flow_for_fg', input=FlowInput(value='comm_day', kind=<FlowInputEnum.HIVE_TABLE: 'hive_table'>), input_date_col=None, tasks=None, output=FlowOutput(value=None, kind=<FlowOutputEnum.SPARK_DATAFRAME: 'spark_dataframe'>), description=''), description=None, features=[Feature(name='comm_count_call_in', feature_id='67', description='', data_type='long', online_data_type='long', created_at='2024-12-30 02:51:

In [9]:
my_fg = FeatureGroup(name="comm_day")
fs = FeatureLookup(source=my_fg)

hist = HistoricalFeatures(lookups=[fs])
spine_dummy_data = pd.DataFrame(
    [
        {"msisdn": "011ezY2Kjs", "app_date": "2019-03-19", "label": 1},
        {"msisdn": "01ViZtJZCj", "app_date": "2019-03-10", "label": 0},
    ]
)
df = hist.using_spine(
    spine=spine_dummy_data, date_col="app_date", keep_cols=["label"]
).to_dataframe()
df.show()

[2024-12-30 02:10:56+0700] WARNING - seeknal | Feature group already exists. Loading the feature group.
[2024-12-30 02:10:56+0700] WARNING - seeknal | Using an existing Flow.
+----------+--------------------+-------------------+------------------+-------------------+---------------------+-----------------+------------------+--------------------+---------------------+----------------------+------------------------+-------------------------+--------------------------+----------------------------+------------------------+-------------------------+---------------------------+----------------------------+-----------------------------+-------------------------------+---------------------+----------------------+------------------------+-------------------------+------------------------+--------------------------+-------------------------+----------------------------+---------------------------+-----------------------+------------------------+----------------------+-----------------------+----

In [10]:
my_fg = FeatureGroup(name="comm_day")
my_fg_two = FeatureGroup(name="comm_day_two")
fs = FeatureLookup(source=my_fg)
fs_two = FeatureLookup(source=my_fg_two)

hist = HistoricalFeatures(lookups=[fs, fs_two])
df = hist.using_latest(
    fetch_strategy=GetLatestTimeStrategy.REQUIRE_ANY
).to_dataframe()
df.show()

[2024-12-30 02:11:07+0700] WARNING - seeknal | Feature group already exists. Loading the feature group.
[2024-12-30 02:11:07+0700] WARNING - seeknal | Using an existing Flow.
[2024-12-30 02:11:07+0700] WARNING - seeknal | Feature group already exists. Loading the feature group.


24/12/30 02:11:09 INFO FeatureStoreSerDe: Duplicate columns are found, will rename intersect columns with prefix feature group name


+----------+--------------------+-------------------+---------------------------+----------------------------+------------------------------+--------------------------+---------------------------+-----------------------------+------------------------------+-------------------------------+---------------------------------+----------------------------------+-----------------------------------+-------------------------------------+---------------------------------+----------------------------------+------------------------------------+-------------------------------------+--------------------------------------+----------------------------------------+------------------------------+-------------------------------+---------------------------------+----------------------------------+---------------------------------+-----------------------------------+----------------------------------+-------------------------------------+------------------------------------+--------------------------------

## 4.2 Serve features to Online Store

In [21]:
my_fg = FeatureGroup(name="comm_day_three")
fs = FeatureLookup(source=my_fg)

user_one = Entity(name="msisdn").get_or_create().set_key_values("05X5wBWKN3")
fillnull = FillNull(value="0.0", dataType="double")
hist = HistoricalFeatures(lookups=[fs], fill_nulls=[fillnull])
hist = hist.using_latest().serve()
abc = hist.get_features(keys=[user_one])
print(abc)

[2024-12-30 16:15:00+0700] WARNING - seeknal | Feature group already exists. Loading the feature group.
[2024-12-30 16:15:00+0700] WARNING - seeknal | Feature group already exists. Loading the feature group.
[2024-12-30 16:15:01+0700] WARNING - seeknal | Using an existing Flow.
[2024-12-30 16:15:01+0700] WARNING - seeknal | Using an existing Flow.
[2024-12-30 16:15:02+0700] WARNING - seeknal | Feature group already exists. Loading the feature group.
[2024-12-30 16:15:02+0700] WARNING - seeknal | Feature group already exists. Loading the feature group.


/Users/fitrakacamarga/project/mta/signal/.venv/lib/python3.11/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `enum` but got `str` with value `'file'` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


[2024-12-30 16:15:02+0700] WARNING - seeknal | Using an existing Flow.
[2024-12-30 16:15:02+0700] WARNING - seeknal | Using an existing Flow.
[{'msisdn': '05X5wBWKN3', 'comm_count_call_in': 0, 'comm_count_call_out': 2, 'comm_count_call_inout': 2, 'comm_count_sms_in': 0, 'comm_count_sms_out': 0, 'comm_count_sms_inout': 0, 'comm_count_callsms_in': 0, 'comm_count_callsms_out': 2, 'comm_count_callsms_inout': 2, 'comm_roamingcount_call_in': 0, 'comm_roamingcount_call_out': 0, 'comm_roamingcount_call_inout': 0, 'comm_roamingcount_sms_in': 0, 'comm_roamingcount_sms_out': 0, 'comm_roamingcount_sms_inout': 0, 'comm_roamingcount_callsms_in': 0, 'comm_roamingcount_callsms_out': 0, 'comm_roamingcount_callsms_inout': 0, 'comm_duration_call_in': 0.0, 'comm_duration_call_out': 227.0, 'comm_duration_call_inout': 227.0, 'comm_callratio_callsms_in': 0.0, 'comm_smsratio_callsms_in': 0.0, 'comm_callratio_callsms_out': 1.0, 'comm_smsratio_callsms_out': 0.0, 'comm_callratio_callsms_inout': 1.0, 'comm_smsrat

In [22]:
my_fg = FeatureGroup(name="comm_day_two")
fs = FeatureLookup(source=my_fg)

user_one = Entity(name="msisdn").get_or_create().set_key_values("05X5wBWKN3")
fillnull = FillNull(value="0.0", dataType="double")
hist = HistoricalFeatures(lookups=[fs], fill_nulls=[fillnull])
online_store = OnlineStore(value=FeatureStoreFileOutput(path="/tmp/online_store"))
online_table = hist.using_latest().serve(
    target=online_store, ttl=timedelta(minutes=1)
)
print(online_table.get_features(keys=[user_one]))

[2024-12-30 16:15:10+0700] WARNING - seeknal | Feature group already exists. Loading the feature group.
[2024-12-30 16:15:10+0700] WARNING - seeknal | Feature group already exists. Loading the feature group.
[2024-12-30 16:15:12+0700] WARNING - seeknal | Feature group already exists. Loading the feature group.
[2024-12-30 16:15:12+0700] WARNING - seeknal | Feature group already exists. Loading the feature group.
[2024-12-30 16:15:12+0700] WARNING - seeknal | Using an existing Flow.
[2024-12-30 16:15:12+0700] WARNING - seeknal | Using an existing Flow.
[2024-12-30 16:15:12+0700] WARNING - seeknal | Refreshing online_table since it is expired based specified ttl.
[2024-12-30 16:15:12+0700] WARNING - seeknal | Refreshing online_table since it is expired based specified ttl.


TypeError: string indices must be integers, not 'str'

In [9]:
abc = spark.read.table("comm_day").withColumnRenamed("day", "event_time")
online_table = OnlineFeatures(
    lookup_key=Entity(name="msisdn").get_or_create(), dataframe=abc
)
abc = online_table.get_features(keys=[{"msisdn": "05X5wBWKN3"}])
print(abc)

[{'comm_count_call_in': 0, 'comm_count_call_out': 2, 'comm_count_call_inout': 2, 'comm_count_sms_in': 0, 'comm_count_sms_out': 0, 'comm_count_sms_inout': 0, 'comm_count_callsms_in': 0, 'comm_count_callsms_out': 2, 'comm_count_callsms_inout': 2, 'comm_roamingcount_call_in': 0, 'comm_roamingcount_call_out': 0, 'comm_roamingcount_call_inout': 0, 'comm_roamingcount_sms_in': 0, 'comm_roamingcount_sms_out': 0, 'comm_roamingcount_sms_inout': 0, 'comm_roamingcount_callsms_in': 0, 'comm_roamingcount_callsms_out': 0, 'comm_roamingcount_callsms_inout': 0, 'comm_duration_call_in': 0.0, 'comm_duration_call_out': 548.0, 'comm_duration_call_inout': 548.0, 'comm_callratio_callsms_in': None, 'comm_smsratio_callsms_in': None, 'comm_callratio_callsms_out': 1.0, 'comm_smsratio_callsms_out': 0.0, 'comm_callratio_callsms_inout': 1.0, 'comm_smsratio_callsms_inout': 0.0, 'comm_inratio_inout_call': 0.0, 'comm_outratio_inout_call': 1.0, 'comm_inratio_inout_sms': None, 'comm_outratio_inout_sms': None, 'comm_inra